<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import data</a></span><ul class="toc-item"><li><span><a href="#Schools-of-interest" data-toc-modified-id="Schools-of-interest-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Schools of interest</a></span></li><li><span><a href="#Check-if-schools-have-the-survey" data-toc-modified-id="Check-if-schools-have-the-survey-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Check if schools have the survey</a></span></li></ul></li><li><span><a href="#Treating-survey-data" data-toc-modified-id="Treating-survey-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Treating survey data</a></span><ul class="toc-item"><li><span><a href="#Rename-categories" data-toc-modified-id="Rename-categories-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Rename categories</a></span></li><li><span><a href="#Tagging-schools" data-toc-modified-id="Tagging-schools-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Tagging schools</a></span></li><li><span><a href="#Model-schools" data-toc-modified-id="Model-schools-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Model schools</a></span></li><li><span><a href="#Risk-schools" data-toc-modified-id="Risk-schools-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Risk schools</a></span></li></ul></li><li><span><a href="#Analysis-of-the-results" data-toc-modified-id="Analysis-of-the-results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analysis of the results</a></span></li></ul></div>

In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, '../')
from config import RAW_PATH, TREAT_PATH, OUTPUT_PATH

import os
from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

### Import data

- Survey w/ Diretors of schools from 2017 (Saeb): http://inep.gov.br/web/guest/microdados (Saeb > 2017)
  - Data: Dados > `TS_DIRETOR.csv`
  - Questions: Questionários > `Questionario do Diretor.pdf`

In [2]:
df_survey = pd.read_csv(RAW_PATH / 'saeb' / 'TS_DIRETOR_2017.csv')
df_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73674 entries, 0 to 73673
Columns: 118 entries, ID_PROVA_BRASIL to TX_RESP_Q111
dtypes: int64(7), object(111)
memory usage: 66.3+ MB


In [3]:
df_survey.head()

,ID_PROVA_BRASIL,ID_UF,ID_MUNICIPIO,ID_ESCOLA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_PREENCHIMENTO_QUESTIONARIO,TX_RESP_Q001,TX_RESP_Q002,TX_RESP_Q003,TX_RESP_Q004,TX_RESP_Q005,TX_RESP_Q006,TX_RESP_Q007,TX_RESP_Q008,TX_RESP_Q009,TX_RESP_Q010,TX_RESP_Q011,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q014,TX_RESP_Q015,TX_RESP_Q016,TX_RESP_Q017,TX_RESP_Q018,TX_RESP_Q019,TX_RESP_Q020,TX_RESP_Q021,TX_RESP_Q022,TX_RESP_Q023,TX_RESP_Q024,TX_RESP_Q025,TX_RESP_Q026,TX_RESP_Q027,TX_RESP_Q028,TX_RESP_Q029,TX_RESP_Q030,TX_RESP_Q031,TX_RESP_Q032,TX_RESP_Q033,TX_RESP_Q034,TX_RESP_Q035,TX_RESP_Q036,TX_RESP_Q037,TX_RESP_Q038,TX_RESP_Q039,TX_RESP_Q040,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,TX_RESP_Q044,TX_RESP_Q045,TX_RESP_Q046,TX_RESP_Q047,TX_RESP_Q048,TX_RESP_Q049,TX_RESP_Q050,TX_RESP_Q051,TX_RESP_Q052,TX_RESP_Q053,TX_RESP_Q054,TX_RESP_Q055,TX_RESP_Q056,TX_RESP_Q057,TX_RESP_Q058,TX_RESP_Q059,TX_RESP_Q060,TX_RESP_Q061,TX_RESP_Q062,TX_RESP_Q063,TX_RESP_Q064,TX_RESP_Q065,TX_RESP_Q066,TX_RESP_Q067,TX_RESP_Q068,TX_RESP_Q069,TX_RESP_Q070,TX_RESP_Q071,TX_RESP_Q072,TX_RESP_Q073,TX_RESP_Q074,TX_RESP_Q075,TX_RESP_Q076,TX_RESP_Q077,TX_RESP_Q078,TX_RESP_Q079,TX_RESP_Q080,TX_RESP_Q081,TX_RESP_Q082,TX_RESP_Q083,TX_RESP_Q084,TX_RESP_Q085,TX_RESP_Q086,TX_RESP_Q087,TX_RESP_Q088,TX_RESP_Q089,TX_RESP_Q090,TX_RESP_Q091,TX_RESP_Q092,TX_RESP_Q093,TX_RESP_Q094,TX_RESP_Q095,TX_RESP_Q096,TX_RESP_Q097,TX_RESP_Q098,TX_RESP_Q099,TX_RESP_Q100,TX_RESP_Q101,TX_RESP_Q102,TX_RESP_Q103,TX_RESP_Q104,TX_RESP_Q105,TX_RESP_Q106,TX_RESP_Q107,TX_RESP_Q108,TX_RESP_Q109,TX_RESP_Q110,TX_RESP_Q111
0,2017,11,1100015,11024666,3,2,1,B,D,D,D,C,B,D,C,E,E,C,E,B,C,H,A,A,G,B,D,B,C,C,C,B,A,A,D,D,B,E,H,A,A,A,A,C,A,F,A,C,C,B,C,D,D,D,D,B,B,B,C,B,B,B,B,B,B,A,B,A,C,C,D,C,C,C,B,B,B,B,A,A,B,B,B,B,A,A,A,A,B,A,B,A,B,A,A,A,B,A,B,B,B,B,B,B,B,B,A,A,B,B,A,A,B,B,A,B,C,C
1,2017,11,1100015,11024682,2,1,1,B,F,C,D,C,C,C,C,E,I,C,I,B,B,G,E,D,G,A,A,B,C,C,C,C,B,E,D,E,B,E,F,A,A,A,A,D,D,E,H,D,E,B,C,C,C,C,B,B,B,B,B,B,B,B,D,B,B,C,A,A,D,D,D,E,C,A,C,A,A,A,A,A,A,B,C,B,A,A,A,A,A,B,B,A,B,B,A,A,A,A,B,B,A,B,B,B,B,B,A,B,B,B,A,A,B,B,A,A,C,A
2,2017,11,1100015,11024828,3,1,1,B,E,C,D,C,B,D,C,E,F,B,F,A,B,D,B,B,D,B,D,B,B,B,C,B,B,B,D,E,C,E,D,C,B,B,B,E,A,F,I,E,E,B,C,C,C,C,C,B,C,C,C,B,C,B,B,B,A,A,A,A,D,D,D,C,D,D,C,C,D,D,B,B,C,C,B,B,A,A,A,A,B,A,B,A,B,A,A,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,A,A,A,B,B,C,C,C
3,2017,11,1100015,11024968,2,1,1,B,E,C,D,E,D,B,C,E,F,C,F,B,C,H,A,A,G,B,E,B,B,B,B,C,B,E,C,D,B,E,A,A,A,B,B,E,A,E,I,E,E,B,C,C,C,C,C,B,B,B,B,B,A,A,B,B,C,A,B,A,D,D,D,C,D,D,C,B,C,D,C,B,C,C,B,B,A,A,A,A,A,B,B,A,B,B,B,C,B,B,B,B,B,B,B,B,B,B,B,B,A,A,A,B,A,A,A,C,C,A
4,2017,11,1100015,11025077,3,1,1,B,D,C,D,C,C,B,C,E,G,C,G,A,C,D,F,A,F,A,A,B,C,B,B,C,A,A,E,B,C,B,H,C,B,B,B,C,A,E,I,E,A,B,D,D,C,C,C,A,B,B,B,B,B,B,C,B,A,B,B,A,D,D,D,D,C,A,B,B,D,B,A,C,A,B,C,B,A,A,A,A,B,A,A,A,B,C,A,C,A,A,B,B,B,B,B,B,B,B,B,B,B,B,A,B,B,B,B,B,C,C


In [4]:
df_survey = df_survey[df_survey['ID_MUNICIPIO'] == 3304557] # schools from Rio de Janeiro city
df_survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 44700 to 45902
Columns: 118 entries, ID_PROVA_BRASIL to TX_RESP_Q111
dtypes: int64(7), object(111)
memory usage: 1.1+ MB


#### Schools of interest

In [5]:
df_schools_ideb = pd.read_csv(OUTPUT_PATH / 'model_neighboors_closest_multiple.csv', index_col=0)
df_schools_ideb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 4560 to 4729
Data columns (total 10 columns):
escola_risco                 122 non-null object
cod_inep_risco               122 non-null int64
ideb_2017_risco              122 non-null float64
nome_escola                  122 non-null object
cod_inep                     122 non-null int64
ideb                         122 non-null float64
pessimo_pra_bom_bin          122 non-null int64
ruim_pra_bom_bin             122 non-null int64
melhora_com_final_bom_bin    122 non-null int64
distancia                    122 non-null float64
dtypes: float64(3), int64(5), object(2)
memory usage: 10.5+ KB


In [6]:
df_schools_ideb.head()

,escola_risco,cod_inep_risco,ideb_2017_risco,nome_escola,cod_inep,ideb,pessimo_pra_bom_bin,ruim_pra_bom_bin,melhora_com_final_bom_bin,distancia
4560,1019064 ESCOLA MUNICIPAL ALVARO VALLE,33146110,5.3,1019043 ESCOLA MUNICIPAL PROFESSORA FLAVIA DOS...,33084971,6.7,0,1,0,1943.246359
4141,0622204 CIEP JOAO DO RIO,33086508,5.4,0622021 ESCOLA MUNICIPAL ISAIAS ALVES,33087121,6.3,0,1,0,80.141361
4405,0625035 ESCOLA MUNICIPAL PREFEITO MARCOS TAMOYO,33139792,5.1,0625021 ESCOLA MUNICIPAL OCTAVIO TARQUINIO DE ...,33087270,6.0,0,1,0,1044.029563
4309,0430002 ESCOLA MUNICIPAL TEOTONIO VILELA,33069450,5.2,0410001 ESCOLA MUNICIPAL EMA NEGRAO DE LIMA,33069247,6.1,0,1,0,1651.566907
4158,0430501 CIEP PRESIDENTE SAMORA MACHEL,33068801,4.3,0410009 ESCOLA MUNICIPAL DILERMANDO CRUZ,33069220,6.2,0,1,0,1335.069098


In [7]:
df_schools_risk = df_schools_ideb[['cod_inep_risco', 'escola_risco', 'ideb_2017_risco']]
df_schools_risk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 4560 to 4729
Data columns (total 3 columns):
cod_inep_risco     122 non-null int64
escola_risco       122 non-null object
ideb_2017_risco    122 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.8+ KB


In [8]:
df_schools_model = df_schools_ideb[['cod_inep', 'nome_escola', 'ideb']].drop_duplicates()
df_schools_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 4560 to 4600
Data columns (total 3 columns):
cod_inep       35 non-null int64
nome_escola    35 non-null object
ideb           35 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


#### Check if schools have the survey

In [9]:
len(df_schools_model[df_schools_model['cod_inep'].isin(df_survey['ID_ESCOLA']) == True]['cod_inep'])

35

In [10]:
len(df_schools_risk[df_schools_risk['cod_inep_risco'].isin(df_survey['ID_ESCOLA']) == True]['cod_inep_risco'])

122

### Treating survey data

#### Rename categories

In [11]:
# import PyPDF2 - o texto do pdf fica muito ruim!
# pdf = open(RAW_PATH / 'saeb' / 'questionarios' / 'Questionario do diretor.pdf', mode='rb')
# read_pdf = PyPDF2.PdfFileReader(pdf)

In [12]:
# df_survey.columns

In [13]:
# {'Q_DIR_SEXO': {'A': 'Masculino', 
#                 'B': 'Feminino'}, 
#  'Q_DIR_IDADE': {}, 
#  'Q_DIR_RACA'}

#### Tagging schools

In [15]:
tag_model = [(idx, 'model') for idx in df_schools_model['cod_inep']]
tag_risk = [(idx, 'risk') for idx in df_schools_risk['cod_inep_risco']]

In [18]:
df_survey['tag'] = df_survey['ID_ESCOLA'].map(dict(tag_model + tag_risk))

{33084971: 'model',
 33087121: 'model',
 33087270: 'model',
 33069247: 'model',
 33069220: 'model',
 33069190: 'model',
 33140081: 'model',
 33086001: 'model',
 33085935: 'model',
 33068364: 'model',
 33072728: 'model',
 33076081: 'model',
 33086494: 'model',
 33070776: 'model',
 33074429: 'model',
 33066248: 'model',
 33084785: 'model',
 33084980: 'model',
 33068321: 'model',
 33063192: 'model',
 33073155: 'model',
 33070717: 'model',
 33070881: 'model',
 33075999: 'model',
 33063184: 'model',
 33073139: 'model',
 33087229: 'model',
 33085013: 'model',
 33085056: 'model',
 33080330: 'model',
 33100551: 'model',
 33080500: 'model',
 33080879: 'model',
 33080925: 'model',
 33080208: 'model',
 33146110: 'risk',
 33086508: 'risk',
 33139792: 'risk',
 33069450: 'risk',
 33068801: 'risk',
 33068739: 'risk',
 33068747: 'risk',
 33069301: 'risk',
 33083169: 'risk',
 33086087: 'risk',
 33086176: 'risk',
 33086060: 'risk',
 33085420: 'risk',
 33085412: 'risk',
 33068372: 'risk',
 33088489: 'ris

#### Model schools

In [11]:
model_survey = df_saeb[df_saeb['ID_ESCOLA'].isin(model_survey)].rename({'ID_ESCOLA': 'cod_inep'})
model_survey.head()

,ID_PROVA_BRASIL,ID_UF,ID_MUNICIPIO,ID_ESCOLA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_PREENCHIMENTO_QUESTIONARIO,TX_RESP_Q001,TX_RESP_Q002,TX_RESP_Q003,TX_RESP_Q004,TX_RESP_Q005,TX_RESP_Q006,TX_RESP_Q007,TX_RESP_Q008,TX_RESP_Q009,TX_RESP_Q010,TX_RESP_Q011,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q014,TX_RESP_Q015,TX_RESP_Q016,TX_RESP_Q017,TX_RESP_Q018,TX_RESP_Q019,TX_RESP_Q020,TX_RESP_Q021,TX_RESP_Q022,TX_RESP_Q023,TX_RESP_Q024,TX_RESP_Q025,TX_RESP_Q026,TX_RESP_Q027,TX_RESP_Q028,TX_RESP_Q029,TX_RESP_Q030,TX_RESP_Q031,TX_RESP_Q032,TX_RESP_Q033,TX_RESP_Q034,TX_RESP_Q035,TX_RESP_Q036,TX_RESP_Q037,TX_RESP_Q038,TX_RESP_Q039,TX_RESP_Q040,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,TX_RESP_Q044,TX_RESP_Q045,TX_RESP_Q046,TX_RESP_Q047,TX_RESP_Q048,TX_RESP_Q049,TX_RESP_Q050,TX_RESP_Q051,TX_RESP_Q052,TX_RESP_Q053,TX_RESP_Q054,TX_RESP_Q055,TX_RESP_Q056,TX_RESP_Q057,TX_RESP_Q058,TX_RESP_Q059,TX_RESP_Q060,TX_RESP_Q061,TX_RESP_Q062,TX_RESP_Q063,TX_RESP_Q064,TX_RESP_Q065,TX_RESP_Q066,TX_RESP_Q067,TX_RESP_Q068,TX_RESP_Q069,TX_RESP_Q070,TX_RESP_Q071,TX_RESP_Q072,TX_RESP_Q073,TX_RESP_Q074,TX_RESP_Q075,TX_RESP_Q076,TX_RESP_Q077,TX_RESP_Q078,TX_RESP_Q079,TX_RESP_Q080,TX_RESP_Q081,TX_RESP_Q082,TX_RESP_Q083,TX_RESP_Q084,TX_RESP_Q085,TX_RESP_Q086,TX_RESP_Q087,TX_RESP_Q088,TX_RESP_Q089,TX_RESP_Q090,TX_RESP_Q091,TX_RESP_Q092,TX_RESP_Q093,TX_RESP_Q094,TX_RESP_Q095,TX_RESP_Q096,TX_RESP_Q097,TX_RESP_Q098,TX_RESP_Q099,TX_RESP_Q100,TX_RESP_Q101,TX_RESP_Q102,TX_RESP_Q103,TX_RESP_Q104,TX_RESP_Q105,TX_RESP_Q106,TX_RESP_Q107,TX_RESP_Q108,TX_RESP_Q109,TX_RESP_Q110,TX_RESP_Q111
44724,2017,33,3304557,33063184,3,1,1,B,E,A,I,E,D,B,A,F,J,C,J,A,B,H,D,D,G,B,E,B,B,C,C,B,A,A,E,E,B,E,NaN,A,A,C,B,E,D,E,H,D,D,B,C,C,C,C,C,C,B,B,B,C,C,B,C,C,A,A,A,A,B,B,C,E,C,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,B,A,B,A,B,A,A,A,B,A,B,B,B,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A
44725,2017,33,3304557,33063192,3,1,1,B,F,A,H,B,B,B,C,E,I,C,I,A,E,E,G,D,G,A,A,A,B,B,B,B,B,C,E,E,B,E,H,A,A,A,A,E,A,A,A,C,C,B,C,D,D,D,D,B,C,D,D,D,A,A,D,B,C,B,C,C,E,E,E,E,E,A,A,A,A,A,A,A,B,A,B,B,A,A,A,A,B,A,B,A,B,A,B,A,B,B,B,B,B,B,B,B,B,B,A,A,A,B,B,A,B,B,A,A,A,A
44772,2017,33,3304557,33066248,3,1,1,B,D,C,D,C,B,B,C,E,K,C,K,B,B,D,G,D,G,B,D,B,C,C,C,B,B,E,E,E,B,E,H,A,A,A,A,E,D,E,I,D,D,B,D,D,D,D,C,C,C,C,D,C,B,B,B,B,A,A,A,A,C,D,D,D,E,D,A,C,D,C,A,A,A,A,B,A,A,A,A,A,B,A,B,A,B,B,A,A,B,B,B,B,B,B,B,B,B,B,A,A,A,A,A,B,A,A,A,A,A,A
44814,2017,33,3304557,33068321,3,1,1,B,D,A,H,E,B,B,C,E,J,C,J,B,E,H,D,D,G,B,C,B,C,C,C,B,B,E,E,E,B,E,F,A,A,A,A,E,A,E,I,B,D,B,D,D,D,D,D,A,B,C,C,C,A,C,C,A,A,A,A,A,D,D,D,C,E,D,C,C,B,B,A,A,A,A,B,A,A,A,A,B,B,A,B,A,B,A,A,A,A,A,B,B,B,B,B,B,B,B,A,A,B,B,A,B,B,B,A,A,A,A
44818,2017,33,3304557,33068364,3,1,1,B,F,C,H,E,B,B,C,E,I,C,I,B,E,G,E,E,G,A,A,B,C,C,NaN,B,A,A,E,E,B,E,H,A,A,A,A,E,A,F,H,C,D,B,C,D,D,D,D,A,B,A,C,B,B,A,C,C,A,A,A,A,C,C,C,C,C,A,A,A,A,A,A,A,A,A,B,B,A,A,A,A,B,A,B,A,B,A,A,A,B,A,B,B,B,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A


In [12]:
model_survey['IN_PREENCHIMENTO_QUESTIONARIO'].value_counts()

1    35
Name: IN_PREENCHIMENTO_QUESTIONARIO, dtype: int64

#### Risk schools

In [13]:
risk_survey = df_saeb[df_saeb['ID_ESCOLA'].isin(risk_survey)].rename({'ID_ESCOLA': 'cod_inep_risco'})
risk_survey.head()

,ID_PROVA_BRASIL,ID_UF,ID_MUNICIPIO,ID_ESCOLA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_PREENCHIMENTO_QUESTIONARIO,TX_RESP_Q001,TX_RESP_Q002,TX_RESP_Q003,TX_RESP_Q004,TX_RESP_Q005,TX_RESP_Q006,TX_RESP_Q007,TX_RESP_Q008,TX_RESP_Q009,TX_RESP_Q010,TX_RESP_Q011,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q014,TX_RESP_Q015,TX_RESP_Q016,TX_RESP_Q017,TX_RESP_Q018,TX_RESP_Q019,TX_RESP_Q020,TX_RESP_Q021,TX_RESP_Q022,TX_RESP_Q023,TX_RESP_Q024,TX_RESP_Q025,TX_RESP_Q026,TX_RESP_Q027,TX_RESP_Q028,TX_RESP_Q029,TX_RESP_Q030,TX_RESP_Q031,TX_RESP_Q032,TX_RESP_Q033,TX_RESP_Q034,TX_RESP_Q035,TX_RESP_Q036,TX_RESP_Q037,TX_RESP_Q038,TX_RESP_Q039,TX_RESP_Q040,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,TX_RESP_Q044,TX_RESP_Q045,TX_RESP_Q046,TX_RESP_Q047,TX_RESP_Q048,TX_RESP_Q049,TX_RESP_Q050,TX_RESP_Q051,TX_RESP_Q052,TX_RESP_Q053,TX_RESP_Q054,TX_RESP_Q055,TX_RESP_Q056,TX_RESP_Q057,TX_RESP_Q058,TX_RESP_Q059,TX_RESP_Q060,TX_RESP_Q061,TX_RESP_Q062,TX_RESP_Q063,TX_RESP_Q064,TX_RESP_Q065,TX_RESP_Q066,TX_RESP_Q067,TX_RESP_Q068,TX_RESP_Q069,TX_RESP_Q070,TX_RESP_Q071,TX_RESP_Q072,TX_RESP_Q073,TX_RESP_Q074,TX_RESP_Q075,TX_RESP_Q076,TX_RESP_Q077,TX_RESP_Q078,TX_RESP_Q079,TX_RESP_Q080,TX_RESP_Q081,TX_RESP_Q082,TX_RESP_Q083,TX_RESP_Q084,TX_RESP_Q085,TX_RESP_Q086,TX_RESP_Q087,TX_RESP_Q088,TX_RESP_Q089,TX_RESP_Q090,TX_RESP_Q091,TX_RESP_Q092,TX_RESP_Q093,TX_RESP_Q094,TX_RESP_Q095,TX_RESP_Q096,TX_RESP_Q097,TX_RESP_Q098,TX_RESP_Q099,TX_RESP_Q100,TX_RESP_Q101,TX_RESP_Q102,TX_RESP_Q103,TX_RESP_Q104,TX_RESP_Q105,TX_RESP_Q106,TX_RESP_Q107,TX_RESP_Q108,TX_RESP_Q109,TX_RESP_Q110,TX_RESP_Q111
44702,2017,33,3304557,33062420,3,1,1,B,D,A,D,B,C,B,A,A,I,C,I,B,B,D,C,C,F,B,D,B,B,B,B,C,B,E,E,E,B,E,D,A,A,A,A,E,A,A,H,C,D,B,C,B,C,C,C,C,B,B,C,C,C,B,B,C,C,A,B,B,C,D,D,C,D,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,A,B,B,B,B,A,A,B,B,B,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A
44717,2017,33,3304557,33063109,3,1,1,B,D,A,H,E,D,B,A,A,I,C,K,B,F,H,E,B,G,B,E,A,A,C,C,B,B,E,E,E,B,E,H,A,A,A,B,D,A,D,I,C,D,B,D,D,D,D,D,B,D,D,C,C,A,D,B,A,A,A,A,A,E,D,E,D,D,C,C,D,A,A,A,A,B,A,A,B,A,A,A,A,B,A,B,A,B,A,B,B,A,B,B,B,B,B,B,B,B,B,A,A,A,A,A,B,B,A,A,C,B,B
44721,2017,33,3304557,33063150,3,1,1,B,F,C,D,E,B,B,NaN,NaN,I,C,I,A,B,G,G,G,G,B,B,B,B,C,C,B,A,A,E,E,B,E,H,A,A,A,B,E,A,A,I,C,C,B,C,D,D,D,D,A,A,B,C,B,A,A,B,A,A,A,A,A,C,D,D,D,D,B,B,B,C,A,A,A,B,A,B,B,A,A,A,A,B,B,B,A,B,A,B,A,A,A,B,B,B,B,B,B,B,B,A,A,A,A,NaN,A,B,B,A,A,C,A
44723,2017,33,3304557,33063176,3,1,1,B,C,B,G,C,B,B,A,A,F,C,F,B,E,F,D,B,F,A,D,A,B,C,C,B,A,A,B,E,B,E,F,A,A,A,A,E,A,A,A,D,D,B,C,C,D,D,D,B,A,A,C,A,A,B,D,A,C,A,B,A,D,D,D,D,D,C,C,D,C,C,B,B,C,B,B,B,A,A,A,A,A,A,B,A,B,A,B,A,A,A,B,B,B,B,B,B,B,B,A,B,B,B,A,B,A,B,A,A,A,A
44770,2017,33,3304557,33066205,3,1,1,B,C,C,D,C,B,B,C,E,I,C,I,B,C,E,B,B,E,B,D,B,B,C,C,B,B,E,E,E,B,E,F,B,C,A,A,E,D,E,A,C,D,B,C,D,C,D,C,B,A,A,B,C,A,B,D,B,B,A,B,B,D,D,D,E,C,A,A,A,A,A,A,A,B,A,B,A,A,A,A,A,B,A,B,B,D,A,B,A,A,A,B,B,B,B,B,B,B,B,A,A,A,A,A,B,A,A,A,C,C,C


In [14]:
risk_survey['IN_PREENCHIMENTO_QUESTIONARIO'].value_counts()

1    122
Name: IN_PREENCHIMENTO_QUESTIONARIO, dtype: int64

### Analysis of the results

* Qual o seu sexo? A: Masculino / B: Feminino

In [15]:
risk_survey['TX_RESP_Q001'].value_counts(normalize=True)

B    0.893443
A    0.106557
Name: TX_RESP_Q001, dtype: float64

In [16]:
model_survey['TX_RESP_Q001'].value_counts(normalize=True)

B    0.914286
A    0.085714
Name: TX_RESP_Q001, dtype: float64

* QUAL É O PERCENTUAL DE PROFESSORES COM VÍNCULO ESTÁVEL NESTA ESCOLA?

In [19]:
risk_survey['TX_RESP_Q028'].value_counts(normalize=True) # E: 91 a 100%

E    0.669421
D    0.214876
C    0.082645
A    0.016529
B    0.016529
Name: TX_RESP_Q028, dtype: float64

In [20]:
model_survey['TX_RESP_Q028'].value_counts(normalize=True) # E: 91 a 100%

E    0.771429
D    0.142857
C    0.057143
B    0.028571
Name: TX_RESP_Q028, dtype: float64

* O CONSELHO ESCOLAR É UM COLEGIADO GERALMENTE CONSTITUÍDO POR REPRESENTANTES DA ESCOLA E DA COMUNIDADE QUE TEM COMO OBJETIVO ACOMPANHAR AS ATIVIDADES ESCOLARES. NESTE ANO, QUANTAS VEZES SE REUNIU O CONSELHO ESCOLAR?

In [21]:
risk_survey['TX_RESP_Q029'].value_counts(normalize=True) # E: 3 ou mais

E    0.926230
D    0.057377
C    0.008197
B    0.008197
Name: TX_RESP_Q029, dtype: float64

In [22]:
model_survey['TX_RESP_Q029'].value_counts(normalize=True) # E: 3 ou mais

E    1.0
Name: TX_RESP_Q029, dtype: float64

* NESTE ANO E NESTA ESCOLA, COMO SE DEU A ELABORAÇÃO DO PROJETO
PEDAGÓGICO?

In [23]:
risk_survey['TX_RESP_Q032'].value_counts(normalize=True) # H: Elaborou-se um modelo próprio e houve discussão com a equipe escolar.

H    0.680328
F    0.254098
D    0.049180
A    0.008197
E    0.008197
Name: TX_RESP_Q032, dtype: float64

In [24]:
model_survey['TX_RESP_Q032'].value_counts(normalize=True) # H: Elaborou-se um modelo próprio e houve discussão com a equipe escolar.

H    0.764706
F    0.147059
D    0.058824
G    0.029412
Name: TX_RESP_Q032, dtype: float64

INDIQUE COM QUAL FREQUÊNCIA SÃO DESENVOLVIDAS AS SEGUINTES ATIVIDADES PARA MINIMIZAR AS FALTAS DOS ALUNOS NESTE ANO E NESTA ESCOLA:

45. Os professores conversam com os alunos para tentar solucionar o problema.
    [Nunca.
    Algumas vezes.
    Frequentemente.
    Sempre ou quase sempre.]

In [25]:
risk_survey['TX_RESP_Q045'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.586777
C    0.388430
B    0.024793
Name: TX_RESP_Q045, dtype: float64

In [26]:
model_survey['TX_RESP_Q045'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.657143
C    0.342857
Name: TX_RESP_Q045, dtype: float64

46. Os pais/responsáveis são avisados por comunicação da escola.

In [27]:
risk_survey['TX_RESP_Q046'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.595041
C    0.388430
B    0.016529
Name: TX_RESP_Q046, dtype: float64

In [28]:
model_survey['TX_RESP_Q046'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.714286
C    0.285714
Name: TX_RESP_Q046, dtype: float64

47. Os pais/responsáveis são chamados à escola para conversar sobre o assunto em reunião de pais.



In [29]:
risk_survey['TX_RESP_Q047'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.595041
C    0.371901
B    0.033058
Name: TX_RESP_Q047, dtype: float64

In [30]:
model_survey['TX_RESP_Q047'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.742857
C    0.228571
A    0.028571
Name: TX_RESP_Q047, dtype: float64

48. Os pais/responsáveis são chamados à escola para conversar sobre o assunto individualmente.


In [31]:
risk_survey['TX_RESP_Q048'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.619835
C    0.322314
B    0.049587
A    0.008264
Name: TX_RESP_Q048, dtype: float64

In [32]:
model_survey['TX_RESP_Q048'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.600000
C    0.371429
B    0.028571
Name: TX_RESP_Q048, dtype: float64


49. A escola envia alguém à casa do aluno.

In [33]:
risk_survey['TX_RESP_Q049'].value_counts(normalize=True) # B: Algumas vezes

B    0.537190
A    0.239669
C    0.123967
D    0.099174
Name: TX_RESP_Q049, dtype: float64

In [34]:
model_survey['TX_RESP_Q049'].value_counts(normalize=True) # B: Algumas vezes

B    0.485714
A    0.285714
C    0.142857
D    0.085714
Name: TX_RESP_Q049, dtype: float64

INDIQUE COM QUAL FREQUÊNCIA VOCÊ DESENVOLVEU AS SEGUINTES ATIVIDADES NESTE ANO E NESTA ESCOLA:

50. Desenvolveu atividades extracurriculares em esporte.

In [35]:
risk_survey['TX_RESP_Q050'].value_counts(normalize=True) # B: Algumas vezes

B    0.438017
C    0.223140
A    0.190083
D    0.148760
Name: TX_RESP_Q050, dtype: float64

In [36]:
model_survey['TX_RESP_Q050'].value_counts(normalize=True) # B: Algumas vezes

B    0.485714
C    0.285714
D    0.142857
A    0.085714
Name: TX_RESP_Q050, dtype: float64

51. Desenvolveu atividades extracurriculares em artes.

In [37]:
risk_survey['TX_RESP_Q051'].value_counts(normalize=True) # B: Algumas vezes

B    0.533333
C    0.225000
D    0.125000
A    0.116667
Name: TX_RESP_Q051, dtype: float64

In [38]:
model_survey['TX_RESP_Q051'].value_counts(normalize=True) # C: Frequentemente

B    0.342857
C    0.342857
A    0.200000
D    0.114286
Name: TX_RESP_Q051, dtype: float64

52. Desenvolveu projetos temáticos (ex: bullying, meio ambiente, desigualdades sociais etc.).

In [39]:
risk_survey['TX_RESP_Q052'].value_counts(normalize=True) # C: Frequentemente

C    0.421488
D    0.297521
B    0.272727
A    0.008264
Name: TX_RESP_Q052, dtype: float64

In [40]:
model_survey['TX_RESP_Q052'].value_counts(normalize=True) # D: Sempre ou quase sempre

D    0.457143
C    0.428571
B    0.114286
Name: TX_RESP_Q052, dtype: float64

53. Neste ano, a escola promoveu eventos para a comunidade.


In [72]:
risk_survey['TX_RESP_Q053'].value_counts() # B: Algumas vezes

B    56
C    32
D    26
A     7
Name: TX_RESP_Q053, dtype: int64

In [89]:
model_survey['TX_RESP_Q053'].value_counts() # C: Frequentemente

C    18
B     9
D     7
A     1
Name: TX_RESP_Q053, dtype: int64


54. Os espaços desta escola são utilizados para eventos promovidos pela comunidade.


In [74]:
risk_survey['TX_RESP_Q054'].value_counts() # A: Nunca

A    84
B    28
C     6
D     2
Name: TX_RESP_Q054, dtype: int64

In [90]:
model_survey['TX_RESP_Q054'].value_counts() # A: Nunca

A    23
B    11
C     1
Name: TX_RESP_Q054, dtype: int64


55. Neste ano, a comunidade colaborou com trabalho voluntário para esta escola
(por exemplo, desenvolvendo atividades, ajudando na manutenção da escola etc.).

In [75]:
risk_survey['TX_RESP_Q055'].value_counts() # A: Nunca

A    63
B    47
C     7
D     3
Name: TX_RESP_Q055, dtype: int64

In [91]:
model_survey['TX_RESP_Q055'].value_counts() # A: Nunca

A    16
B    14
C     5
Name: TX_RESP_Q055, dtype: int64